In [1]:
from compas_pr2d import PR2DModel
from compas_pr2d import wall


In [2]:

print("Constructing the model ...")
model = PR2DModel()
print("Constructing the wall geometry ...")

nl = 10
nh = 4
length = 10
height = 6

wall_geometry = wall.construct_wall(length, height, nl, nh)
print("Feeding the geometry to the model ...")

model.from_polygons(wall_geometry)
# model.display_mesh(bindex=True, nindex=True)  # Keep only for when boundary edges needs to be chosen individually

boundary_edges = [
    1,
    7,
    11,
    15,
    19,
    23,
    27,
    31,
    35,
    39,
]  # Optional list of boundary edge IDs.
# Otherwise, boundary edges will be automatically detected.
# But works only for edges with one block.

print("Assembling Matrices:")

model.set_boundary_edges(display=True, boundary_edges=boundary_edges)
disp_data = [
    [66, (0.3, 0.25), (0.0, 0.00)],
    [67, (0.25, 0.2), (0.0, 0.00)],
    [68, (0.2, 0.15), (0.0, 0.00)],
    [69, (0.15, 0.1), (0.0, 0.00)],
    [70, (0.1, 0.05), (0.0, 0.00)],
    [71, (0.05, 0.0), (0.0, 0.00)],
]  # [Interface ID, normal displacement, tangential displacement]

# disp_data = [
#     [66, (0.0, 0.0), (0.05, 0.005)],
#     [67, (0.0, 0.0), (0.05, 0.005)],
#     [68, (0.0, 0.0), (0.05, 0.005)],
#     [69, (0.0, 0.0), (-0.05, -0.005)],
#     [70, (0.0, 0.0), (-0.05, -0.005)],
#     [71, (0.0, 0.0), (-0.05, -0.005)],
# ]
# model.set_force()  # for now no loads, just self-weight which is default in the model (i will add as gravity to all blocks for now only)
print("Assigning BCs to RHS vectors...")
model.assign_bc(disp_data)
# print("Solving the system...")
model.solve()

model.display_results()  # postprocesses and displays results


Constructing the model ...
Constructing the wall geometry ...
Feeding the geometry to the model ...
Assembling Matrices:
Assembling boundary elements for edge: (0, 66, (0, 3))
Boundary edge rows: 132, 133
Assembling boundary elements for edge: (1, 67, (3, 5))
Boundary edge rows: 134, 135
Assembling boundary elements for edge: (2, 68, (5, 7))
Boundary edge rows: 136, 137
Assembling boundary elements for edge: (3, 69, (7, 9))
Boundary edge rows: 138, 139
Assembling boundary elements for edge: (4, 70, (9, 11))
Boundary edge rows: 140, 141
Assembling boundary elements for edge: (5, 71, (11, 13))
Boundary edge rows: 142, 143
Assembling boundary elements for edge: (6, 72, (13, 15))
Boundary edge rows: 144, 145
Assembling boundary elements for edge: (7, 73, (15, 17))
Boundary edge rows: 146, 147
Assembling boundary elements for edge: (8, 74, (17, 19))
Boundary edge rows: 148, 149
Assembling boundary elements for edge: (9, 75, (19, 21))
Boundary edge rows: 150, 151
Assigning BCs to RHS vectors

qt.qpa.drawing: Layer-backing is always enabled.  QT_MAC_WANTS_LAYER/_q_mac_wantsLayer has no effect.


In [114]:
import numpy as np
import pyvista as pv

A = model.mesh
Results = model.results

nodes = np.zeros((A.number_of_vertices(),3),dtype=float)
print(nodes.shape)

# Version that minimizes dictionary lookups
v = A.vertex
n = A.number_of_vertices()
for i in range(n):
    p = v[i]
    nodes[i] = (p["x"], p["y"], p["z"])
# --------------------------------------------


faces = np.zeros((A.number_of_faces(),5),dtype=int)
m = A.number_of_faces()

F = A.face_vertices

for j in range(m):
    face = F(j)
    
    faces[j] = [len(face)] + face
connectivity = np.hstack(faces)
print(connectivity)
# print(f"Node matrix is {nodes}")
# print(f"Face matrix is {faces}")

grid = pv.UnstructuredGrid(connectivity, np.full(A.number_of_faces(),pv.CellType.POLYGON), nodes)

plt = pv.Plotter()
U = Results.U.reshape(m,3)
Uy = U[:,1]
Uy_np = np.asarray(Uy,dtype=float)
plt.add_mesh(grid, scalars=Uy_np, show_edges=True, cmap="jet", opacity=1.0)
# plt.show_grid()
plt.disable_stereo_render()
plt.enable_surface_point_picking()
plt.show()

INFO:trame_server.controller:trigger(trigger__78)
INFO:trame_server.controller:trigger(trigger__79)
INFO:trame_server.controller:trigger(P_0x3b96e07f0_21Camera)
INFO:trame_server.controller:trigger(P_0x3b96e07f0_21AnimateStart)
INFO:trame_server.controller:trigger(P_0x3b96e07f0_21AnimateStop)
INFO:trame_client.widgets.core:js_key = class
INFO:trame_client.widgets.core:js_key = style
INFO:trame_client.widgets.core:js_key = fluid
INFO:trame_client.widgets.core:js_key = class
INFO:trame_client.widgets.core:before: class = { 'rounded-circle': !P_0x3b96e07f0_21_show_ui }
INFO:trame_server.utils.namespace:(prefix=None) token {
INFO:trame_server.state:has({ => {) = False
INFO:trame_server.utils.namespace:(prefix=None) token  
INFO:trame_server.state:has(  =>  ) = False
INFO:trame_server.utils.namespace:(prefix=None) token '
INFO:trame_server.state:has(' => ') = False
INFO:trame_server.utils.namespace:(prefix=None) token rounded
INFO:trame_server.state:has(rounded => rounded) = False
INFO:tram

(55, 3)
[ 4  0  1  2  3  4  3  2  4  5  4  5  4  6  7  4  7  6  8  9  4  9  8 10
 11  4 11 10 12 13  4 13 12 14 15  4 15 14 16 17  4 17 16 18 19  4 19 18
 20 21  4  1 22 23  2  4  2 23 24  4  4  4 24 25  6  4  6 25 26  8  4  8
 26 27 10  4 10 27 28 12  4 12 28 29 14  4 14 29 30 16  4 16 30 31 18  4
 18 31 32 20  4 22 33 34 23  4 23 34 35 24  4 24 35 36 25  4 25 36 37 26
  4 26 37 38 27  4 27 38 39 28  4 28 39 40 29  4 29 40 41 30  4 30 41 42
 31  4 31 42 43 32  4 33 44 45 34  4 34 45 46 35  4 35 46 47 36  4 36 47
 48 37  4 37 48 49 38  4 38 49 50 39  4 39 50 51 40  4 40 51 52 41  4 41
 52 53 42  4 42 53 54 43]


INFO:trame_server.utils.namespace:(prefix=None) token P_0x3b96e07f0_21_edge_visibility
INFO:trame_server.state:has(P_0x3b96e07f0_21_edge_visibility => P_0x3b96e07f0_21_edge_visibility) = True
INFO:trame_server.utils.namespace:(prefix=None) translated P_0x3b96e07f0_21_edge_visibility
INFO:trame_server.utils.namespace:(prefix=None) token  
INFO:trame_server.state:has(  =>  ) = False
INFO:trame_server.utils.namespace:(prefix=None) token ?
INFO:trame_server.state:has(? => ?) = False
INFO:trame_server.utils.namespace:(prefix=None) token  
INFO:trame_server.state:has(  =>  ) = False
INFO:trame_server.utils.namespace:(prefix=None) token '
INFO:trame_server.state:has(' => ') = False
INFO:trame_server.utils.namespace:(prefix=None) token on
INFO:trame_server.state:has(on => on) = False
INFO:trame_server.utils.namespace:(prefix=None) token '
INFO:trame_server.state:has(' => ') = False
INFO:trame_server.utils.namespace:(prefix=None) token  
INFO:trame_server.state:has(  =>  ) = False
INFO:trame_se

Widget(value='<iframe src="http://localhost:60639/index.html?ui=P_0x3b96e07f0_21&reconnect=auto" class="pyvist…